# Einstein constraints on a Boyer-Lindquist slice of Kerr spacetime

This notebook demonstrates a few capabilities of SageMath in computations regarding hypersurfaces of a pseudo-Riemannian manifold. The corresponding tools have been developed in the framework of the  [SageManifolds](https://sagemanifolds.obspm.fr) project.

In [1]:
version()

'SageMath version 10.4.beta6, Release Date: 2024-05-12'

First we set up the notebook to display maths with LaTeX rendering:

In [2]:
%display latex

We ask for running computations in parallel on 8 threads:

In [3]:
Parallelism().set(nproc=8)

## Kerr spacetime $(M,g)$

The mass parameter $m$ and spin parameter $a$ of Kerr solution:

In [4]:
m, a = var('m a', domain='real')
assume(a<m)

The Kerr spacetime declared as a 4-dimensional Lorentzian manifold:

In [5]:
M = Manifold(4, 'M', structure='Lorentzian')
print(M)

4-dimensional Lorentzian manifold M


On $M$, we consider the *"rational-polynomial" coordinates* $(t,r,y,\phi)$ inherited from the standard **Boyer-Lindquist coordinates** $(t,r,\theta,\phi)$ via $y=\cos\theta$:

In [6]:
BL.<t,r,y,ph> = M.chart(r't r:(0,+oo) y:(-1,1) ph:(0,2*pi):\phi') 
BL

Chart (M, (t, r, y, ph))

We focus on the exterior of the Kerr black hole, on which $\Delta := r^2 - 2 mr + a^2 > 0$: 

In [7]:
assume(a^2 -2*m*r+ r^2 > 0)

The Kerr metric:

In [8]:
g = M.metric()
rho2 = r^2 + (a*y)^2
Delta = r^2 -2*m*r + a^2
g[0,0] = -(1-2*m*r/rho2)
g[0,3] = -2*a*m*r*(1 - y^2)/rho2
g[1,1], g[2,2] = rho2/Delta, rho2/(1 - y^2)
g[3,3] = (r^2 + a^2 + 2*m*r*a^2*(1 - y^2)/rho2)*(1 - y^2)
g.display()

g = (2*m*r/(a^2*y^2 + r^2) - 1) dt⊗dt + 2*(y^2 - 1)*a*m*r/(a^2*y^2 + r^2) dt⊗dph + (a^2*y^2 + r^2)/(a^2 - 2*m*r + r^2) dr⊗dr - (a^2*y^2 + r^2)/(y^2 - 1) dy⊗dy + 2*(y^2 - 1)*a*m*r/(a^2*y^2 + r^2) dph⊗dt + (2*(y^2 - 1)*a^2*m*r/(a^2*y^2 + r^2) - a^2 - r^2)*(y^2 - 1) dph⊗dph

As a test, let us check that $g$ is a solution of the **vacuum Einstein equation**:

In [9]:
g.ricci().display()

Ric(g) = 0

## Spacelike hypersurface $(\Sigma,\gamma)$

We declare $\Sigma$ as a 3-dimensional (immersed) submanifold of $M$ via the keyword `ambient`:

In [10]:
S = Manifold(3, 'S', latex_name=r'\Sigma', ambient=M, 
             structure="Riemannian", start_index=1)
print(S)

3-dimensional Riemannian submanifold S immersed in the 4-dimensional Lorentzian manifold M


In [11]:
S

3-dimensional Riemannian submanifold S immersed in the 4-dimensional Lorentzian manifold M

We consider $\Sigma$ to be a constant Boyer-Lindquist coordinate $t$ hypersurface. Natural coordinates on $\Sigma$ are then the Boyer-Lindquist coordinates $(r,y,\phi)$:

In [12]:
BLS = S.chart(r'r:(0,+oo) y:(-1,1) ph:(0,2*pi):\phi')
BLS

Chart (S, (r, y, ph))

At this stage, the user atlas of $\Sigma$ contains a single chart:

In [13]:
S.atlas()

[Chart (S, (r, y, ph))]

The embedding $\Phi: \Sigma \to M$:

In [14]:
Phi = S.diff_map(M, {(BLS, BL): (t, r, y, ph)}, name='Phi',
                 latex_name=r'\Phi')
Phi.display()

Phi: S → M
   (r, y, ph) ↦ (t, r, y, ph) = (t, r, y, ph)

The partial inverse:

In [15]:
Phi_inv = M.diff_map(S, {(BL, BLS): (r, y, ph)})
Phi_inv.display()

M → S
   (t, r, y, ph) ↦ (r, y, ph) = (r, y, ph)

In [16]:
Phi_inv*Phi == S.identity_map()

True

$t$ as a scalar field on $M$:

In [17]:
t_scalar = M.scalar_field(t)
t_scalar.display()

M → ℝ
(t, r, y, ph) ↦ t

We declare that $\Phi$ is the embedding of $\Sigma$ in $M$ and that $\Sigma$ is a level
surface of $t$:

In [18]:
S.set_embedding(Phi, inverse=Phi_inv, var=t,
                t_inverse = {t: t_scalar})
print(S)

3-dimensional Riemannian submanifold S embedded in the 4-dimensional Lorentzian manifold M


For some computations, $\Sigma$ is considered as belonging to a foliation of $M$ by
a family of hypersurfaces $(\Sigma_t)_{t\in\mathbb{R}}$. A chart of $M$ adapted to the
foliation is computed by the method `adapted_chart`. In the present case, it is trivial, 
since $t$ is one of the coordinate of the default chart of $M$:


In [19]:
S.adapted_chart()

[Chart (M, (r_M, y_M, ph_M, t_M))]

In [20]:
M.atlas()

[Chart (M, (t, r, y, ph)), Chart (M, (r_M, y_M, ph_M, t_M))]

### The Riemannian metric $\gamma$ induced by $g$ on $\Sigma$

$\gamma$ is the first fundamental form of $\Sigma$:

In [21]:
gam = S.first_fundamental_form()
gam.display()

gamma = (a^2*y^2 + r^2)/(a^2 - 2*m*r + r^2) dr⊗dr - (a^2*y^2 + r^2)/(y^2 - 1) dy⊗dy - ((a^4 - 2*a^2*m*r + a^2*r^2)*y^4 - 2*a^2*m*r - a^2*r^2 - r^4 - (a^4 - 4*a^2*m*r - r^4)*y^2)/(a^2*y^2 + r^2) dph⊗dph

It is actually the pullback of $g$ by the embedding $\Phi$:

In [22]:
gam == Phi.pullback(g)

True

An alias of `first_fundamental_form` is `induced_metric`:

In [23]:
gam is S.induced_metric()

True

The Levi-Civita connection $D$ associated with $\gamma$:

In [24]:
D = gam.connection(name='D')
print(D)

Levi-Civita connection D associated with the Riemannian metric gamma on the 3-dimensional Riemannian submanifold S embedded in the 4-dimensional Lorentzian manifold M


In [25]:
D.display()

Gam^r_r,r = (a^2*r - m*r^2 + (a^2*m - a^2*r)*y^2)/(a^2*r^2 - 2*m*r^3 + r^4 + (a^4 - 2*a^2*m*r + a^2*r^2)*y^2) 
Gam^r_r,y = a^2*y/(a^2*y^2 + r^2) 
Gam^r_y,r = a^2*y/(a^2*y^2 + r^2) 
Gam^r_y,y = (a^2*r - 2*m*r^2 + r^3)/(a^2*y^4 - (a^2 - r^2)*y^2 - r^2) 
Gam^r_ph,ph = (a^4*m*r^2 - 2*a^2*m^2*r^3 + a^2*m*r^4 - a^2*r^5 + 2*m*r^6 - r^7 - (a^6*m + 3*a^4*m*r^2 - a^4*r^3 - (a^6 + 2*a^4*m^2)*r)*y^6 + (2*a^6*m + 5*a^4*m*r^2 - 3*a^2*m*r^4 + 2*a^2*r^5 + (a^4 - 2*a^2*m^2)*r^3 - (a^6 + 4*a^4*m^2)*r)*y^4 - (a^6*m - 2*a^4*m^2*r + 3*a^4*m*r^2 - 2*a^2*m*r^4 + a^2*r^5 + 2*m*r^6 - r^7 + 2*(a^4 - 2*a^2*m^2)*r^3)*y^2)/(a^6*y^6 + 3*a^4*r^2*y^4 + 3*a^2*r^4*y^2 + r^6) 
Gam^y_r,r = (a^2*y^3 - a^2*y)/(a^2*r^2 - 2*m*r^3 + r^4 + (a^4 - 2*a^2*m*r + a^2*r^2)*y^2) 
Gam^y_r,y = r/(a^2*y^2 + r^2) 
Gam^y_y,r = r/(a^2*y^2 + r^2) 
Gam^y_y,y = -(a^2 + r^2)*y/(a^2*y^4 - (a^2 - r^2)*y^2 - r^2) 
Gam^y_ph,ph = -((a^6 - 2*a^4*m*r + a^4*r^2)*y^7 - (a^6 - 2*a^4*m*r - a^4*r^2 + 4*a^2*m*r^3 - 2*a^2*r^4)*y^5 + (2*a^4*m*r - 2*a^4*r^2 + 8*a^2*m*r^3 - a^2*r^4 + r^6)*y^3 - (2*a^4*m*r + 4*a^2*m*r^3 + a^2*r^4 + r^6)*y)/(a^6*y^6 + 3*a^4*r^2*y^4 + 3*a^2*r^4*y^2 + r^6) 
Gam^ph_r,ph = -(a^2*m*r^2 - r^5 + (a^4*m - a^4*r)*y^4 - (a^4*m + a^2*m*r^2 + 2*a^2*r^3)*y^2)/(2*a^2*m*r^3 + a^2*r^4 + r^6 + (a^6 - 2*a^4*m*r + a^4*r^2)*y^4 + 2*(a^4*m*r + a^4*r^2 - a^2*m*r^3 + a^2*r^4)*y^2) 
Gam^ph_y,ph = ((a^6 - 2*a^4*m*r + a^4*r^2)*y^5 + 2*(a^4*r^2 - 2*a^2*m*r^3 + a^2*r^4)*y^3 + (2*a^4*m*r + 4*a^2*m*r^3 + a^2*r^4 + r^6)*y)/((a^6 - 2*a^4*m*r + a^4*r^2)*y^6 - 2*a^2*m*r^3 - a^2*r^4 - r^6 - (a^6 - 4*a^4*m*r - a^4*r^2 + 2*a^2*m*r^3 - 2*a^2*r^4)*y^4 - (2*a^4*m*r + 2*a^4*r^2 - 4*a^2*m*r^3 + a^2*r^4 - r^6)*y^2) 
Gam^ph_ph,r = -(a^2*m*r^2 - r^5 + (a^4*m - a^4*r)*y^4 - (a^4*m + a^2*m*r^2 + 2*a^2*r^3)*y^2)/(2*a^2*m*r^3 + a^2*r^4 + r^6 + (a^6 - 2*a^4*m*r + a^4*r^2)*y^4 + 2*(a^4*m*r + a^4*r^2 - a^2*m*r^3 + a^2*r^4)*y^2) 
Gam^ph_ph,y = ((a^6 - 2*a^4*m*r + a^4*r^2)*y^5 + 2*(a^4*r^2 - 2*a^2*m*r^3 + a^2*r^4)*y^3 + (2*a^4*m*r + 4*a^2*m*r^3 + a^2*r^4 + r^6)*y)/((a^6 - 2*a^4*m*r + a^4*r^2)*y^6 - 2*a^2*m*r^3 - a^2*r^4 - r^6 - (a^6 - 4*a^4*m*r - a^4*r^2 + 2*a^2*m*r^3 - 2*a^2*r^4)*y^4 - (2*a^4*m*r + 2*a^4*r^2 - 4*a^2*m*r^3 + a^2*r^4 - r^6)*y^2)

The unit normal to $\Sigma$ (actually to the foliation $(\Sigma_t)_{t\in\mathbb{R}}$):

In [26]:
n = S.normal()
print(n)
n.display()

Vector field n on the 4-dimensional Lorentzian manifold M


n = sqrt(2*a^2*m*r + a^2*r^2 + r^4 + (a^4 - 2*a^2*m*r + a^2*r^2)*y^2)*sqrt(a^2*y^2 + r^2)*sqrt(a^2 - 2*m*r + r^2)/(a^2*r^2 - 2*m*r^3 + r^4 + (a^4 - 2*a^2*m*r + a^2*r^2)*y^2) ∂/∂t + 2*sqrt(a^2*y^2 + r^2)*sqrt(a^2 - 2*m*r + r^2)*a*m*r/(sqrt(2*a^2*m*r + a^2*r^2 + r^4 + (a^4 - 2*a^2*m*r + a^2*r^2)*y^2)*(a^2*r^2 - 2*m*r^3 + r^4 + (a^4 - 2*a^2*m*r + a^2*r^2)*y^2)) ∂/∂ph

Let us check that $n$ is a unit timelike vector:

In [27]:
g(n,n).expr()

-1

## The second fundamental form $K$:

In [28]:
K = S.second_fundamental_form()
print(K)

Field of symmetric bilinear forms K on the 3-dimensional Riemannian submanifold S embedded in the 4-dimensional Lorentzian manifold M


In [29]:
K.display()

K = (a^3*m*r^2 + 3*a*m*r^4 + (a^5*m - a^3*m*r^2)*y^4 - (a^5*m + 3*a*m*r^4)*y^2)*sqrt(a^2*y^2 + r^2)/((a^4*y^4 + 2*a^2*r^2*y^2 + r^4)*sqrt(2*a^2*m*r + a^2*r^2 + r^4 + (a^4 - 2*a^2*m*r + a^2*r^2)*y^2)*sqrt(a^2 - 2*m*r + r^2)) dr⊗dph - 2*(a^3*m*r*y^3 - a^3*m*r*y)*sqrt(a^2*y^2 + r^2)*sqrt(a^2 - 2*m*r + r^2)/((a^4*y^4 + 2*a^2*r^2*y^2 + r^4)*sqrt(2*a^2*m*r + a^2*r^2 + r^4 + (a^4 - 2*a^2*m*r + a^2*r^2)*y^2)) dy⊗dph + (a^3*m*r^2 + 3*a*m*r^4 + (a^5*m - a^3*m*r^2)*y^4 - (a^5*m + 3*a*m*r^4)*y^2)*sqrt(a^2*y^2 + r^2)/((a^4*y^4 + 2*a^2*r^2*y^2 + r^4)*sqrt(2*a^2*m*r + a^2*r^2 + r^4 + (a^4 - 2*a^2*m*r + a^2*r^2)*y^2)*sqrt(a^2 - 2*m*r + r^2)) dph⊗dr - 2*(a^3*m*r*y^3 - a^3*m*r*y)*sqrt(a^2*y^2 + r^2)*sqrt(a^2 - 2*m*r + r^2)/((a^4*y^4 + 2*a^2*r^2*y^2 + r^4)*sqrt(2*a^2*m*r + a^2*r^2 + r^4 + (a^4 - 2*a^2*m*r + a^2*r^2)*y^2)) dph⊗dy

In [30]:
K.display_comp()

K_r,ph = (a^3*m*r^2 + 3*a*m*r^4 + (a^5*m - a^3*m*r^2)*y^4 - (a^5*m + 3*a*m*r^4)*y^2)*sqrt(a^2*y^2 + r^2)/((a^4*y^4 + 2*a^2*r^2*y^2 + r^4)*sqrt(2*a^2*m*r + a^2*r^2 + r^4 + (a^4 - 2*a^2*m*r + a^2*r^2)*y^2)*sqrt(a^2 - 2*m*r + r^2)) 
K_y,ph = -2*(a^3*m*r*y^3 - a^3*m*r*y)*sqrt(a^2*y^2 + r^2)*sqrt(a^2 - 2*m*r + r^2)/((a^4*y^4 + 2*a^2*r^2*y^2 + r^4)*sqrt(2*a^2*m*r + a^2*r^2 + r^4 + (a^4 - 2*a^2*m*r + a^2*r^2)*y^2)) 
K_ph,r = (a^3*m*r^2 + 3*a*m*r^4 + (a^5*m - a^3*m*r^2)*y^4 - (a^5*m + 3*a*m*r^4)*y^2)*sqrt(a^2*y^2 + r^2)/((a^4*y^4 + 2*a^2*r^2*y^2 + r^4)*sqrt(2*a^2*m*r + a^2*r^2 + r^4 + (a^4 - 2*a^2*m*r + a^2*r^2)*y^2)*sqrt(a^2 - 2*m*r + r^2)) 
K_ph,y = -2*(a^3*m*r*y^3 - a^3*m*r*y)*sqrt(a^2*y^2 + r^2)*sqrt(a^2 - 2*m*r + r^2)/((a^4*y^4 + 2*a^2*r^2*y^2 + r^4)*sqrt(2*a^2*m*r + a^2*r^2 + r^4 + (a^4 - 2*a^2*m*r + a^2*r^2)*y^2))

The metric dual $K^i_{\ \, j} = \gamma^{ik} K_{kj}$:

In [31]:
Ku = K.up(gam, 0)  # 0 = first index of K
print(Ku)

Tensor field of type (1,1) on the 3-dimensional Riemannian submanifold S embedded in the 4-dimensional Lorentzian manifold M


In [32]:
Ku.display()

(a^3*m*r^2 + 3*a*m*r^4 + (a^5*m - a^3*m*r^2)*y^4 - (a^5*m + 3*a*m*r^4)*y^2)*sqrt(a^2 - 2*m*r + r^2)/((a^4*y^4 + 2*a^2*r^2*y^2 + r^4)*sqrt(2*a^2*m*r + a^2*r^2 + r^4 + (a^4 - 2*a^2*m*r + a^2*r^2)*y^2)*sqrt(a^2*y^2 + r^2)) ∂/∂r⊗dph + 2*(a^3*m*r*y^5 - 2*a^3*m*r*y^3 + a^3*m*r*y)*sqrt(a^2 - 2*m*r + r^2)/((a^4*y^4 + 2*a^2*r^2*y^2 + r^4)*sqrt(2*a^2*m*r + a^2*r^2 + r^4 + (a^4 - 2*a^2*m*r + a^2*r^2)*y^2)*sqrt(a^2*y^2 + r^2)) ∂/∂y⊗dph + (a^3*m*r^2 + 3*a*m*r^4 - (a^5*m - a^3*m*r^2)*y^2)*sqrt(a^2*y^2 + r^2)/((2*a^2*m*r^3 + a^2*r^4 + r^6 + (a^6 - 2*a^4*m*r + a^4*r^2)*y^4 + 2*(a^4*m*r + a^4*r^2 - a^2*m*r^3 + a^2*r^4)*y^2)*sqrt(2*a^2*m*r + a^2*r^2 + r^4 + (a^4 - 2*a^2*m*r + a^2*r^2)*y^2)*sqrt(a^2 - 2*m*r + r^2)) ∂/∂ph⊗dr + 2*sqrt(a^2*y^2 + r^2)*sqrt(a^2 - 2*m*r + r^2)*a^3*m*r*y/((2*a^2*m*r^3 + a^2*r^4 + r^6 + (a^6 - 2*a^4*m*r + a^4*r^2)*y^4 + 2*(a^4*m*r + a^4*r^2 - a^2*m*r^3 + a^2*r^4)*y^2)*sqrt(2*a^2*m*r + a^2*r^2 + r^4 + (a^4 - 2*a^2*m*r + a^2*r^2)*y^2)) ∂/∂ph⊗dy

The trace $K := K^i_{\ \, i}$:

In [33]:
trK = Ku.trace()
print(trK)

Scalar field zero on the 3-dimensional Riemannian submanifold S embedded in the 4-dimensional Lorentzian manifold M


We have $K=0$:

In [34]:
trK.expr()

0

i.e. $\Sigma$ is a **maximal hypersurface** of $(M,g)$.

## Hamiltonian constraint

The vacuum Hamiltonian constraint equation is 
$$ R + K^2 -K_{ij} K^{ij} = 0 $$
where $R = \mathrm{tr}_\gamma \mathbf{R} = \gamma^{ij} R_{ij}$ is the Ricci scalar of $\gamma$.
Let us first evaluate $R$:

In [35]:
R = gam.ricci_scalar()
print(R)
R.display()

Scalar field r(gamma) on the 3-dimensional Riemannian submanifold S embedded in the 4-dimensional Lorentzian manifold M


r(gamma): S → ℝ
   (r, y, ph) ↦ 2*(a^6*m^2*r^4 + 6*a^4*m^2*r^6 + 9*a^2*m^2*r^8 - (a^10*m^2 - 6*a^8*m^2*r^2 + 8*a^6*m^3*r^3 - 3*a^6*m^2*r^4)*y^6 + (a^10*m^2 - 8*a^8*m^2*r^2 + 16*a^6*m^3*r^3 - 3*a^6*m^2*r^4 - 6*a^4*m^2*r^6)*y^4 + (2*a^8*m^2*r^2 - 8*a^6*m^3*r^3 - a^6*m^2*r^4 - 9*a^2*m^2*r^8)*y^2)/(4*a^4*m^2*r^8 + 4*a^4*m*r^9 + a^4*r^10 + 4*a^2*m*r^11 + 2*a^2*r^12 + r^14 + (a^14 - 4*a^12*m*r - 4*a^10*m*r^3 + a^10*r^4 + 2*(a^12 + 2*a^10*m^2)*r^2)*y^10 + (4*a^12*m*r - 12*a^10*m*r^3 - 16*a^8*m*r^5 + 5*a^8*r^6 + 2*(5*a^10 + 6*a^8*m^2)*r^4 + (5*a^12 - 8*a^10*m^2)*r^2)*y^8 + 2*(2*a^10*m^2*r^2 + 8*a^10*m*r^3 - 4*a^8*m*r^5 - 12*a^6*m*r^7 + 5*a^6*r^8 + 2*(5*a^8 + 3*a^6*m^2)*r^6 + (5*a^10 - 12*a^8*m^2)*r^4)*y^6 + 2*(6*a^8*m^2*r^4 + 12*a^8*m*r^5 + 4*a^6*m*r^7 - 8*a^4*m*r^9 + 5*a^4*r^10 + 2*(5*a^6 + a^4*m^2)*r^8 + (5*a^8 - 12*a^6*m^2)*r^6)*y^4 + (12*a^6*m^2*r^6 + 16*a^6*m*r^7 + 12*a^4*m*r^9 + 10*a^4*r^10 - 4*a^2*m*r^11 + 5*a^2*r^12 + (5*a^6 - 8*a^4*m^2)*r^8)*y^2)

In [36]:
R.expr().factor()

-2*(a^8*y^4 - 6*a^6*r^2*y^4 + 8*a^4*m*r^3*y^4 - 3*a^4*r^4*y^4 + 2*a^6*r^2*y^2 - 8*a^4*m*r^3*y^2 + 6*a^2*r^6*y^2 + a^4*r^4 + 6*a^2*r^6 + 9*r^8)*a^2*m^2*(y + 1)*(y - 1)/((a^4*y^2 - 2*a^2*m*r*y^2 + a^2*r^2*y^2 + 2*a^2*m*r + a^2*r^2 + r^4)^2*(a^2*y^2 + r^2)^3)

The term $K_{ij} K^{ij}$:

In [37]:
Kuu = Ku.up(gam, 1)
KK = K['_ij']*Kuu['^ij']
print(KK)
KK.display()

Scalar field on the 3-dimensional Riemannian submanifold S embedded in the 4-dimensional Lorentzian manifold M


S → ℝ
(r, y, ph) ↦ 2*(a^6*m^2*r^4 + 6*a^4*m^2*r^6 + 9*a^2*m^2*r^8 - (a^10*m^2 - 6*a^8*m^2*r^2 + 8*a^6*m^3*r^3 - 3*a^6*m^2*r^4)*y^6 + (a^10*m^2 - 8*a^8*m^2*r^2 + 16*a^6*m^3*r^3 - 3*a^6*m^2*r^4 - 6*a^4*m^2*r^6)*y^4 + (2*a^8*m^2*r^2 - 8*a^6*m^3*r^3 - a^6*m^2*r^4 - 9*a^2*m^2*r^8)*y^2)/(4*a^4*m^2*r^8 + 4*a^4*m*r^9 + a^4*r^10 + 4*a^2*m*r^11 + 2*a^2*r^12 + r^14 + (a^14 - 4*a^12*m*r - 4*a^10*m*r^3 + a^10*r^4 + 2*(a^12 + 2*a^10*m^2)*r^2)*y^10 + (4*a^12*m*r - 12*a^10*m*r^3 - 16*a^8*m*r^5 + 5*a^8*r^6 + 2*(5*a^10 + 6*a^8*m^2)*r^4 + (5*a^12 - 8*a^10*m^2)*r^2)*y^8 + 2*(2*a^10*m^2*r^2 + 8*a^10*m*r^3 - 4*a^8*m*r^5 - 12*a^6*m*r^7 + 5*a^6*r^8 + 2*(5*a^8 + 3*a^6*m^2)*r^6 + (5*a^10 - 12*a^8*m^2)*r^4)*y^6 + 2*(6*a^8*m^2*r^4 + 12*a^8*m*r^5 + 4*a^6*m*r^7 - 8*a^4*m*r^9 + 5*a^4*r^10 + 2*(5*a^6 + a^4*m^2)*r^8 + (5*a^8 - 12*a^6*m^2)*r^6)*y^4 + (12*a^6*m^2*r^6 + 16*a^6*m*r^7 + 12*a^4*m*r^9 + 10*a^4*r^10 - 4*a^2*m*r^11 + 5*a^2*r^12 + (5*a^6 - 8*a^4*m^2)*r^8)*y^2)

Finally the Hamiltonian constraint:

In [38]:
Ham = R + trK^2 - KK
print(Ham)
Ham.display()

Scalar field on the 3-dimensional Riemannian submanifold S embedded in the 4-dimensional Lorentzian manifold M


S → ℝ
(r, y, ph) ↦ 0

In [39]:
Ham.expr()

0

Hence the Hamiltonian constraint is fulfilled. 

## The momentum constraint

In vacuum, the momentum constraint is
$$ D_j K^j_{\ \, i} - D_i K = 0 $$

Given that $D_k K^j_{\ \, i}$ is the tensor $(DK)^j_{\ \, ik}$, we evaluate the momentum constraint 
as follows:

In [40]:
mom = D(Ku)['^j_{ij}'] - D(trK)
print(mom)

1-form on the 3-dimensional Riemannian submanifold S embedded in the 4-dimensional Lorentzian manifold M


In [41]:
mom.display()

0

Hence the momentum constraint is fulfilled. 

*Remark:* an alternative writting for `D(Ku)['^j_{ij}']` is `D(Ku).trace(0,2)`:

In [42]:
mom == D(Ku).trace(0,2) - D(trK)

True